In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import numpy as np
import seaborn as sns
plt.rcParams["figure.figsize"]=[15,5]

In [2]:
raw_sp = pd.read_csv('screenprinter_plus_vaf.csv', usecols =['timestamp', 'data.A1'])
raw_sp.dropna(axis=0, inplace=True)
raw_sp.sort_values(by=['timestamp'])
raw_sp.reset_index(inplace=True)
raw_sp['timestamp'] =  pd.to_datetime(raw_sp['timestamp'])

In [3]:
test_sp=raw_sp

In [4]:
#Get boards from Screenprinter
test_sp=test_sp[['data.A1','timestamp']]
# events=scipy.signal.find_peaks(test_sp['data.A1'], height=(2.22, 5), width=1)

In [5]:
test_sp['sum']=pd.Series.to_frame(test_sp['data.A1'].rolling(6, center=True).sum())
test_sp['sum_forcleaning']=pd.Series.to_frame(test_sp['data.A1'].rolling(12, center=True).sum())

In [6]:
%matplotlib qt

In [7]:
plt.ylabel="Screenprinter Current (A)"
plt.plot(test_sp['timestamp'], test_sp['data.A1'])


In [8]:
# plt.plot(test_sp['timestamp'], test_sp['sum'], 'blue')
plt.plot(test_sp['timestamp'], test_sp['sum_forcleaning']/100, 'orange')
# plt.plot(test_sp['timestamp'], test_sp['data.A1']*5, 'orange')
plt.plot(test_sp['timestamp'], [11]*test_sp.shape[0])

In [8]:
plt.plot(test_sp['timestamp'], test_sp['sum'])

In [14]:
test_sp['idle']=2

In [61]:
test_sp['binary_printing']=0
test_sp['binary_printing'][test_sp['sum']>=11] = 1



/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
sptimethresh=12
printing_delays_raw=scipy.signal.find_peaks(test_sp['sum'], distance=sptimethresh, width=1)
printing_delays_raw_df=pd.DataFrame({"sample_number":printing_delays_raw[0], "working_time":printing_delays_raw[1]['right_bases']-printing_delays_raw[1]['left_bases']})
cleaning_delays_raw=scipy.signal.find_peaks(test_sp['sum_forcleaning'], height=60, distance=sptimethresh, width=1)
cleaning_delays_raw_df=pd.DataFrame({"sample_number":cleaning_delays_raw[0], "working_time":cleaning_delays_raw[1]['widths']})

In [75]:
# okman=pd.Series(printing_delays_raw[1]['right_bases']-printing_delays_raw[1]['left_bases'])
# upperquartile=okman.quantile(0.75)
# okman[okman<=upperquartile]

In [10]:
printing_delays_raw_df

,sample_number,working_time
0,17,22
1,37,7
2,52,26
3,67,134
4,85,21
5,118,26
6,134,66
7,151,22
8,170,6
9,184,24


In [11]:
for index, i in enumerate(printing_delays_raw[0]):
    test_sp.ix[int(printing_delays_raw[1]['left_ips'][index]) : int(printing_delays_raw[1]['right_ips'][index]), 'idle']=0

for index, i in enumerate(cleaning_delays_raw[0]):
    test_sp.ix[int(cleaning_delays_raw[1]['left_ips'][index]) : int(cleaning_delays_raw[1]['right_ips'][index]), 'idle']=0

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [12]:
idle_delays_raw=scipy.signal.find_peaks(test_sp['idle'], height=.5, width=1)
idle_delays_raw_df=pd.DataFrame({"sample_number":idle_delays_raw[0], "working_time":idle_delays_raw[1]['widths']})

In [13]:
SP_events=pd.DataFrame({"timestamp":test_sp.iloc[printing_delays_raw_df.sample_number].timestamp, "event":1, "working_time":printing_delays_raw_df.working_time.tolist()})
cleanings=pd.DataFrame({"timestamp":test_sp.iloc[cleaning_delays_raw_df.sample_number].timestamp, "event":2, "working_time":cleaning_delays_raw_df.working_time.tolist()})

SP_events.reset_index(inplace=True)
cleanings.reset_index(inplace=True)

In [14]:
idles=pd.DataFrame({"timestamp":test_sp.iloc[idle_delays_raw_df.sample_number].timestamp, "event":0, "working_time":idle_delays_raw_df.working_time.tolist()})
idles.reset_index(inplace=True)

In [15]:
for x, row in SP_events.iterrows():
    SP_events.ix[x,'energy']=test_sp.ix[int(printing_delays_raw[1]['left_ips'][x]):int(printing_delays_raw[1]['right_ips'][x]), 'data.A1'].sum()*230/3600000
# for index, x in enumerate(printing_delays_raw[0]):
#     SP_events.ix[index,'energy']=np.trapz(test_sp.ix[int(printing_delays_raw[1]['left_ips'][index]):int(printing_delays_raw[1]['right_ips'][index]), 'data.A1'].tolist()*230/3600000)

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [16]:
for x, row in cleanings.iterrows():
    cleanings.ix[x,'energy']=test_sp.ix[int(cleaning_delays_raw[1]['left_ips'][x]):int(cleaning_delays_raw[1]['right_ips'][x]), 'data.A1'].sum()*230/3600000

# for index,x in enumerate(cleaning_delays_raw[0]):
#     cleanings.ix[index,'energy']=np.trapz(test_sp.ix[int(cleaning_delays_raw[1]['left_ips'][index]):int(cleaning_delays_raw[1]['right_ips'][index]), 'data.A1'].tolist()*230)
    

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [17]:
for x, row in idles.iterrows():
    idles.ix[x,'energy']=test_sp.ix[int(idle_delays_raw[1]['left_ips'][x]):int(idle_delays_raw[1]['right_ips'][x]), 'data.A1'].sum()*230/3600000

In [18]:
SP_events.index=SP_events.timestamp
SP_events.drop('timestamp', axis=1, inplace=True)
SP_events.drop('index', axis=1, inplace=True)

cleanings.index=cleanings.timestamp
cleanings.drop('timestamp', axis=1, inplace=True)
cleanings.drop('index', axis=1, inplace=True)

idles.index=idles.timestamp
idles.drop('timestamp', axis=1, inplace=True)
idles.drop('index', axis=1, inplace=True)


SP_events=SP_events.append(cleanings)
SP_events=SP_events.append(idles)

/home/raghu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [19]:
# sns.distplot(SP_events.working_time[SP_events['event']==1])
# plt.title('SP Printing Times Histogram')

In [20]:
# sns.distplot(SP_events.working_time[SP_events['event']==2])
# plt.title('SP Cleaning Times Histogram')

In [78]:
print('Printing time Mode: ', SP_events.working_time[SP_events['event']==1].mean())

Printing time Mode:  6.751731138503205


In [27]:
print('Cleaning time Mode: ', SP_events.working_time[SP_events['event']==2].mode().mean())

Cleaning time Mode:  12.27074807053289


In [ ]:
SP_events[SP_events.event==1].shape[0]

In [ ]:
cleaning_delays_raw[0][0]

In [ ]:
# plt.plot(test_sp.timestamp, test_sp.detection)
# plt.plot(test_sp.timestamp, test_sp['sum'])

In [ ]:
SP1spower=SP_events[SP_events.event==1]['energy']*3600000/SP_events[SP_events.event==1]['working_time']
SP1spower.mean()

In [ ]:
SP0spower=SP_events[SP_events.event==0]['energy']*3600000/SP_events[SP_events.event==0]['working_time']
SP0spower.mean()

In [ ]:
SP_events[SP_events.event==1].shape[0]

In [ ]:
SP_events[SP_events.event==2].shape[0]

In [34]:
plt.stem(SP_events.index, SP_events.event==1, 'red')
plt.stem(SP_events.index, SP_events.event==2, 'green')


<StemContainer object of 3 artists>